```
Copyright 2021 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Support Vector Machine on SUSY Dataset

## Background 

This is a classification problem to distinguish between a signal process which produces supersymmetric particles and a background process which does not.

## Source

Daniel Whiteson daniel '@' uci.edu, Assistant Professor, Physics & Astronomy, Univ. of California Irvine.

In this example, we download the dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php).

## Goal
The goal of this notebook is to illustrate how Snap ML can accelerate training of a support vector machine model on this dataset.

## Code

In [1]:
cd ../../

/home/aan/snapml-examples-2/snapml-examples/examples


In [2]:
CACHE_DIR='cache-dir'

In [3]:
import numpy as np
import time
from datasets import Susy
from sklearn.svm import LinearSVC
from snapml import SupportVectorMachine as SnapSupportVectorMachine
from sklearn.metrics import accuracy_score as score

In [4]:
dataset = Susy(cache_dir=CACHE_DIR)
X_train, X_test, y_train, y_test = dataset.get_train_test_split()

Reading binary Susy dataset (cache) from disk.


In [5]:
print("Number of examples: %d" % (X_train.shape[0]))
print("Number of features: %d" % (X_train.shape[1]))
print("Number of classes:  %d" % (len(np.unique(y_train))))

Number of examples: 3750000
Number of features: 18
Number of classes:  2


In [6]:
model = LinearSVC(loss="hinge", fit_intercept=False, random_state=42)
t0 = time.time()
model.fit(X_train, y_train)
t_fit_sklearn = time.time()-t0
score_sklearn = score(y_test, model.predict(X_test))
print("Training time (sklearn): %6.2f seconds" % (t_fit_sklearn))
print("Accuracy score (sklearn): %.4f" % (score_sklearn))

/home/aan/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training time (sklearn):   7.73 seconds
Accuracy score (sklearn): 0.7872


In [8]:
model = SnapSupportVectorMachine(n_jobs=4, fit_intercept=False, random_state=42)
t0 = time.time()
model.fit(X_train, y_train)
t_fit_snapml = time.time()-t0
score_snapml = score(y_test, model.predict(X_test))
print("Training time (snapml): %6.2f seconds" % (t_fit_snapml))
print("Accuracy score (snapml): %.4f" % (score_snapml))

Training time (snapml):   3.02 seconds
Accuracy score (snapml): 0.7872


In [9]:
speed_up = t_fit_sklearn/t_fit_snapml
score_diff = (score_snapml-score_sklearn)/score_sklearn
print("Speed-up:                %.1f x" % (speed_up))
print("Relative diff. in score: %.4f" % (score_diff))

Speed-up:                2.6 x
Relative diff. in score: -0.0000


## Disclaimer

Performance results always depend on the hardware and software environment. 

Information regarding the environment that was used to run this notebook are provided below:

In [10]:
import utils
environment = utils.get_environment()
for k,v in environment.items():
    print("%15s: %s" % (k, v))

       platform: Linux-4.15.0-140-generic-x86_64-with-debian-buster-sid
      cpu_count: 16
   cpu_freq_min: 1200.0
   cpu_freq_max: 3200.0
   total_memory: 62.82522201538086
 snapml_version: 1.7.1
sklearn_version: 0.24.1


# Record Statistics

Finally, we record the enviroment and performance statistics for analysis outside of this standalone notebook.

In [12]:
import scrapbook as sb
sb.glue("result", {
    'dataset': dataset.name,
    'n_examples_train': X_train.shape[0],
    'n_examples_test': X_test.shape[0],
    'n_features': X_train.shape[1],
    'n_classes': len(np.unique(y_train)),
    'model': type(model).__name__,
    'score': score.__name__,
    't_fit_sklearn': t_fit_sklearn,
    'score_sklearn': score_sklearn,
    't_fit_snapml': t_fit_snapml,
    'score_snapml': score_snapml,
    'score_diff': score_diff,
    'speed_up': speed_up,
    **environment,
})